# Queries with and without Azure OpenAI

## Set up variables

In [1]:
import os
import urllib
import requests
from collections import OrderedDict
from IPython.display import display, HTML
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.embeddings import HuggingFaceEmbeddings

from app.embeddings import OpenAIEmbeddings
from app.prompts import STUFF_PROMPT, REFINE_PROMPT, REFINE_QUESTION_PROMPT
from app.credentials import (
    DATASOURCE_CONNECTION_STRING,
    AZURE_SEARCH_API_VERSION,
    AZURE_SEARCH_ENDPOINT,
    AZURE_SEARCH_KEY,
    COG_SERVICES_NAME,
    COG_SERVICES_KEY,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY,
    AZURE_OPENAI_API_VERSION
)

In [2]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': AZURE_SEARCH_KEY}
params = {'api-version': AZURE_SEARCH_API_VERSION}

## Multi-Index Search queries

In [3]:
# Index that we are going to query (from Notebook 01 and 02)
index1_name = "cogsrch-snowflake-index-files"
indexes = [index1_name]

In [4]:
QUESTION = "What is Snowflake?" 
# This questions is interesting since CLP means something in Computer science and means something different in medical field 

### Search on both indexes individually and aggragate results

Note: In order to standarize the indexes we are setting 4 mandatory fields to be present on each index: id, title, content, pages, language. These fields must be present in each index so that each document can be treated the same along the code.

In [5]:
agg_search_results = []

for index in indexes:
    url = AZURE_SEARCH_ENDPOINT + '/indexes/'+ index + '/docs'
    url += '?api-version={}'.format(AZURE_SEARCH_API_VERSION)
    url += '&search={}'.format(QUESTION)
    url += '&select=*'
    url += '&$top=10'  # You can change this to anything you need/want
    url += '&queryLanguage=en-us'
    url += '&queryType=semantic'
    url += '&semanticConfiguration=my-semantic-config'
    url += '&$count=true'
    url += '&speller=lexicon'
    url += '&answers=extractive|count-3'
    url += '&captions=extractive|highlight-false'

    resp = requests.get(url, headers=headers)
    print(url)
    print(resp.status_code)

    search_results = resp.json()
    agg_search_results.append(search_results)
    print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

https://azure-cog-search-cstevuxaqrxcm.search.windows.net/indexes/cogsrch-snowflake-index-files/docs?api-version=2021-04-30-Preview&search=What is Snowflake?&select=*&$top=10&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-false
200
Results Found: 3, Results Returned: 3


### Display the top results (from both searches) based on the score

In [6]:
display(HTML('<h4>Top Answers</h4>'))

for search_results in agg_search_results:
    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(result['score']) + '</h5>'))
            display(HTML(result['text']))
            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

file_content = OrderedDict()

for search_results in agg_search_results:
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 0.4: # Show results that are at least 10% of the max possible score=4
            display(HTML('<h5>' + str(result['title']) + '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;score: '+ str(result['@search.rerankerScore']) + '</h5>'))
            display(HTML(result['@search.captions'][0]['text']))
            file_content[result['id']]={
                                    "title": result['title'],
                                    "chunks": result['pages'],
                                    "language": result['language'], 
                                    "caption": result['@search.captions'][0]['text'],
                                    "score": result['@search.rerankerScore'],
                                    "location": result['metadata_storage_path']                  
                                }

# Using Azure OpenAI

Of course we want OpenAI to give a better answer, so we instead of sending these results, we send the content of the documents of the search result articles to OpenAI and lets GPT model give the answer.

The problem is that the content of the search result files is or can be very lengthy, more than the allowed tokens allowed by the GPT Azure OpenAI models. So what we need to do is to split in chunks, vectorize and do a vector semantic search. 

Notice that **the documents chunks are already done in Azure Search**. file_content dictionary (created in the cell above) contains the pages (chunks) of each document. So we dont really need to chunk them again, each doc page for sure will fit on the max tokens limit of the completions LLM and of the embedding LLM.

We will use a genius library call LangChain that wraps a lot of boiler plate code.

In [7]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

In [8]:
docs = []
for key,value in file_content.items():
    for page in value["chunks"]:
        docs.append(Document(page_content=page, metadata={"source": value["location"]}))
        
print("Number of chunks:",len(docs))

Number of chunks: 30


Depending of the amount of chunks/pages returned from the search result, which is very related to the size of the documents returned
we pick the embedding model that give us fast results. <br>The logic is, if there is less than 50 chunks (of 5000 chars each) to vectorize then we use 
OpenAI models which currently don't offer batch processing, but if there is more than 50 chunks we use a BERT based in-memory model that processes in batches and in parallel (it is recommended a VM of at least 4 cores).

For more information on in-memory models that you can use, see [HERE](https://www.sbert.net/docs/pretrained_models.html)

In [9]:
# Select the Embedder model
if len(docs) < 50:
    # OpenAI models are accurate but slower
    embedder = OpenAIEmbeddings(document_model_name="text-embedding-ada-002", query_model_name="text-embedding-ada-002") 
else:
    # Bert based models are faster (3x-10x) but not as great in accuracy as OpenAI models
    # Since this repo supports Multiple languages we need to use a multilingual model. 
    # But if English only is the requirement, use "multi-qa-MiniLM-L6-cos-v1"
    # The fastest english model is "all-MiniLM-L12-v2"
    if random.choice(list(file_content.items()))[1]["language"] == "en":
        embedder = HuggingFaceEmbeddings(model_name = 'multi-qa-MiniLM-L6-cos-v1')
    else:
        embedder = HuggingFaceEmbeddings(model_name = 'distiluse-base-multilingual-cased-v2')

In [10]:
embedder

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, document_model_name='text-embedding-ada-002', query_model_name='text-embedding-ada-002', openai_api_key=None)

In [11]:
%%time
if(len(docs)>1):
    db = FAISS.from_documents(docs, embedder)
else:
    print("No results Found")

CPU times: user 139 ms, sys: 28.1 ms, total: 167 ms
Wall time: 3.48 s


In [12]:
docs_db = db.similarity_search(QUESTION, k=4)

At this point we already have the most similar chunks (in order of relevance given by the in-memory vector cosine similarity search) in docs_db

### Now we use GPT-3.5(Turbo) using map-reduce chain in order to stay within the limits of the allow model's token count

for more information on the different types of prompts for these chains please see here:

https://github.com/hwchase17/langchain/tree/master/langchain/chains/question_answering

In [13]:
# Make sure you have the deployment named "gpt-35-turbo" for the model "gpt-35-turbo (0301)". 
# Use "gpt-4" if you have it available.
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0.9, max_tokens=500)
chain = load_qa_with_sources_chain(llm, chain_type="map_reduce", return_intermediate_steps=True)

In [14]:
%%time
response = chain({"input_documents": docs_db, "question": QUESTION}, return_only_outputs=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (2082 > 1024). Running this sequence through the model will result in indexing errors


CPU times: user 839 ms, sys: 100 ms, total: 939 ms
Wall time: 18.7 s


In [20]:
answer = response['output_text']

display(HTML('<h4>Azure OpenAI ChatGPT Answer:</h4>'))

print(answer.split("Source:")[0])
print("Sources:")
print(answer.split("Source:")[1].replace(' ',' ').split(","))

Snowflake is an enterprise-ready data warehouse as a service that is cloud-based, highly scalable, elastic, and supports columnar storage and processing of semi-structured data. It offers automatic type inference and columnar storage for schema-less data, encrypted data import and export, encryption of table data, role-based access control, and two-factor authentication. 
Sources:
[' https://depprocureformstorage.blob.core.windows.net/snowflake-data/Snowflake%20Internals.pdf and https://depprocureformstorage.blob.core.windows.net/snowflake-data/Snowflake_basic%20overview.pdf']


In [21]:
# Uncomment if you want to inspect the results from each top similar chunk (k=4 by default)
response['intermediate_steps']

['\n\n•Snowflake is an enterprise-ready data warehouse as a service\n•Novel multi-cluster, shared-data architecture\n•Highly elastic and available\n•Semi-structured and schema-less data at the speed of relational data\n•Pure SaaS experience',
 '•The Snowflake Elastic Data Warehouse, or “Snowflake”\n•Multi-tenant, transactional, secure, highly scalable, elastic\n•Implemented from scratch (no Hadoop, Postgres etc.)\n•Currently runs on AWS and Azure \n•Serves tens of millions of queries per day over hundreds petabytes of data\n•1000+ active customers, growing fast',
 'Snowflake is a cloud-based enterprise data warehousing service that supports self-describing, compact binary serialization designed for fast key-value lookup, comparison, and hashing. It offers columnar storage and processing of semi-structured data, automatic type inference and columnar storage for schema-less data (VARIANT), encrypted data import and export, encryption of table data using NIST 800-57 compliant hierarchical

# Summary
##### This answer is way better than taking just the result from Azure Cognitive Search. So the summary is:
- Azure Cognitive Search give us the top results (context)
- Azure OpenAI takes these results and understand the content and uses it as context to give the best answer
- Best of two worlds!